In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/34 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N20.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N1.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/49 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N15.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/No18.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/31 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 6.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/3 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 5.jpeg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 1.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/26 no.jpg
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N26.JPG
/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/N5.jpg
/kaggle/input/brain-mri-images-for-b

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split

falseGroup = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/"
trueGroup  = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/"

# Get the list of file paths
false_images = [os.path.join(falseGroup, fname) for fname in os.listdir(falseGroup)]
true_images = [os.path.join(trueGroup, fname) for fname in os.listdir(trueGroup)]

# Create labels
false_labels = [0] * len(false_images)  # Label '0' for no tumor
true_labels = [1] * len(true_images)    # Label '1' for tumor present

# Combine the lists of file paths and labels
file_paths = false_images + true_images
labels = false_labels + true_labels

# Split into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Define a function to load and preprocess images
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, tf.cast(label, tf.float32)

# Create training and validation datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(load_image).batch(32).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=len(X_train))

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(load_image).batch(32, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=len(X_val))


In [7]:
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

def create_resnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the ResNet50 model
resnet_model = create_resnet_model()

# Train the model
history = resnet_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate on the validation dataset
val_loss, val_accuracy = resnet_model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# You can repeat the same process for other models (EfficientNet, VGG16, etc.)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.5178 - loss: 0.9825 - val_accuracy: 0.6078 - val_loss: 1.0016
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.6125 - loss: 0.9466 - val_accuracy: 0.4314 - val_loss: 0.7499
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.5753 - loss: 0.8371 - val_accuracy: 0.6078 - val_loss: 0.7588
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.6569 - loss: 0.7830 - val_accuracy: 0.6471 - val_loss: 0.6861
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.5488 - loss: 0.7600 - val_accuracy: 0.5882 - val_loss: 0.6736
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accuracy: 0.6230 - loss: 0.7978 - val_accuracy: 0.6471 - val_loss: 0.7291
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.5909 - loss: 0.7941 - val_accuracy: 0.5882 - val_loss: 0.6648
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.6097 - loss: 0.7163

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import tensorflow as tf
import pandas as pd

# Model creation functions
def create_resnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_vgg_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mobilenet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_custom_cnn(input_shape=(224, 224, 3), num_classes=1):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model evaluation function
def evaluate_model(model, train_dataset, val_dataset):
    # Start the training timer
    start_time = time.time()
    
    # Train the model
    model.fit(train_dataset, epochs=10, validation_data=val_dataset)
    
    # Stop the timer for training time
    train_time = time.time() - start_time
    
    # Predict on validation data
    y_pred = model.predict(val_dataset)
    y_pred = (y_pred > 0.5).astype(int).flatten()
    
    # Collect all true labels from val_dataset
    y_true = tf.concat([labels for _, labels in val_dataset], axis=0).numpy().flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    
    # Calculate inference time
    inference_start = time.time()
    _ = model.predict(val_images)
    inference_time = (time.time() - inference_start) / len(val_images)
    
    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "auc": auc,
            "train_time": train_time,
            "inference_time": inference_time
           }

# Prepare to run evaluation on all models
model_functions = {
    # 'ResNet50': create_resnet_model,
    # 'VGG16': create_vgg_model,
    'EfficientNetB0': create_efficientnet_model,
    'MobileNetV2': create_mobilenet_model,
    'Custom CNN': create_custom_cnn
}

results = {}

# Iterate through the models and evaluate each one
for model_name, model_function in model_functions.items():
    print(f"Training and evaluating {model_name}...")
    model = model_function()  # Create the model
    results[model_name] = evaluate_model(model, train_dataset, val_dataset)

# Display the results
df_results = pd.DataFrame(results).T  # Transpose for easier viewing
print(df_results)  # Display the DataFrame in console

Training and evaluating EfficientNetB0...
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.5488 - loss: 0.6965 - val_accuracy: 0.6078 - val_loss: 0.6704
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5751 - loss: 0.6961 - val_accuracy: 0.6078 - val_loss: 0.6708
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6207 - loss: 0.6657 - val_accuracy: 0.6078 - val_loss: 0.6697
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6464 - loss: 0.6485 - val_accuracy: 0.6078 - val_loss: 0.6705
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5894 - loss: 0.6814 - val_accuracy: 0.6078 - val_loss: 0.6698
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5441 - loss: 0.7121 - val_accuracy: 0.6078 - val_loss: 0.6714
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6886 - loss: 0.6521 - val_accuracy: 0.6078 - val_loss: 0.6713
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7002 - loss: 0.6252 - val_ac

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6958 - loss: 0.6224 - val_accuracy: 0.7255 - val_loss: 0.6157
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.7686 - loss: 0.4930 - val_accuracy: 0.7647 - val_loss: 0.5941
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8390 - loss: 0.4264 - val_accuracy: 0.7255 - val_loss: 0.6190
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8655 - loss: 0.3099 - val_accuracy: 0.7647 - val_loss: 0.5743
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.8794 - loss: 0.2905 - val_accuracy: 0.7843 - val_loss: 0.6753
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9014 - loss: 0.2350 - val_accuracy: 0.7843 - val_loss: 0.6950
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9166 - loss: 0.2120 - val_accuracy: 0.8235 - val_loss: 0.6385
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9638 - loss: 0.1173 - val_accuracy: 0.8039 - val_loss: 0.9109
Epoch 9/